This file contains the code for paper [**Easy Transfer Learning By Exploiting Intra-domain Structures**](https://arxiv.org/pdf/1904.01376.pdf) published at **IEEE International Conference on Multimedia & Expo (ICME) 2019.**

In [88]:
import pulp
import time
import scipy
import scipy.io
import scipy.stats
import numpy as np

## Intra-domain Programming

Explanation for this function.

In [89]:
def get_class_center(Xs, Ys):
    class_center = []
    classes = np.unique(Ys)

    for c in classes:
        mask = Ys == c
        Xi = Xs[mask.flatten()]
        hc = np.mean(Xi, axis=0)
        class_center.append(hc)

    return np.asarray(class_center)

Explanation for this function.

In [90]:
def get_distance_matrix(Xt, class_center):
    nt = Xt.shape[0]
    C = class_center.shape[0]

    D = np.zeros((C, nt))

    for i, hc in enumerate(class_center):
        for j, xt in enumerate(Xt):
            d = np.linalg.norm(xt - hc)
            D[i][j] = d

    return D

Explanation for this function.

In [91]:
def solve_LP(C, nt, Dcj):

    Dcj = abs(Dcj)
    model = pulp.LpProblem('Cost_minimizing_problem', pulp.LpMinimize)
    Mcj = pulp.LpVariable.dicts('Probabality',
                                ((i, j) for i in range(C) for j in range(nt)),
                                lowBound=0,
                                upBound=1,
                                cat='continous')
    # Objective function
    model += (pulp.lpSum(Dcj[(i, j)] * Mcj[(i, j)] for i in range(C)
                         for j in range(nt)))

    # Constarints
    for j in range(nt):
        model += pulp.lpSum([Mcj[(i, j)] for i in range(C)]) == 1

    for i in range(C):
        model += pulp.lpSum([Mcj[(i, j)] for j in range(nt)]) >= 1

    # Solve the problem
    model.solve()
    pulp.LpStatus[model.status]
    output = [[Mcj[i, j].varValue for i in range(C)] for j in range(nt)]

    return np.array(output)

Explanation for this function.

In [92]:
def intra_domain_programming(Xs, Ys, Xt, Yt):
    C = len(np.unique(Ys))
    nt = len(Yt)

    class_center = get_class_center(Xs, Ys)
    D = get_distance_matrix(Xt, class_center)
    Mcj = solve_LP(C, nt, D)

    y_pred = np.argmax(Mcj, axis=1)
    acc = np.mean(y_pred == Yt.flatten())

    return y_pred, acc

## Intra_domain Alignment

In [93]:
def intra_domain_Alignment(Xs, Xt):
    cov_src = np.ma.cov(Xs.T)
    cov_tar = np.ma.cov(Xt.T)
       
    Es = np.eye(Xs.shape[1])
    Et = np.eye(Xt.shape[1])
    
    Cs = np.sqrt(np.linalg.inv(cov_src + Es))
    Ct = np.sqrt(cov_tar + Et)
    
    coral = np.dot(Cs,Ct)
    
    Xs_new = np.dot(Xs,coral) 
    return Xs_new

## EasyTL

In [94]:
def EasyTL(Xs, Ys, Xt, Yt):
    Xs_new = intra_domain_Alignment(Xs, Xt)
    y_pred, acc = intra_domain_programming(Xs, Ys, Xt, Yt)
    
    return y_pred, acc

## Experiments

### Amazon Review dataset

In [95]:
datadir = 'Datasets/amazon_review/'
domain = ['books', 'dvd', 'elec', 'kitchen']

for i in range(len(domain)):
    for j in range(len(domain)):
        if i == j:
            continue
        print("{} - {}".format(domain[i], domain[j]))

        mat1 = scipy.io.loadmat(datadir + '/{}_400.mat'.format(domain[i]))
        mat2 = scipy.io.loadmat(datadir + '/{}_400.mat'.format(domain[j]))

        Xs = mat1['fts']
        Ys = mat1['labels']

        Xt = mat1['fts']
        Yt = mat1['labels']

        Xs[np.isnan(Xs)] = 0
        Xt[np.isnan(Xt)] = 0

        t0 = time.time()
        _, Acc = EasyTL(Xs, Ys, Xt, Yt)
        t1 = time.time()

        print("Time Elapsed: {:.2f} sec".format(t1 - t0))
        print('EasyTL Acc: {:.1f} %'.format(Acc * 100))

books - dvd


C:\Users\maryam\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  
C:\Users\maryam\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


Time Elapsed: 1.10 sec
EasyTL Acc: 69.5 %
books - elec
Time Elapsed: 1.12 sec
EasyTL Acc: 69.5 %
books - kitchen
Time Elapsed: 1.15 sec
EasyTL Acc: 69.5 %
dvd - books
Time Elapsed: 1.02 sec
EasyTL Acc: 73.8 %
dvd - elec
Time Elapsed: 1.09 sec
EasyTL Acc: 73.8 %
dvd - kitchen
Time Elapsed: 1.12 sec
EasyTL Acc: 73.8 %
elec - books
Time Elapsed: 1.15 sec
EasyTL Acc: 78.2 %
elec - dvd
Time Elapsed: 1.20 sec
EasyTL Acc: 78.2 %
elec - kitchen
Time Elapsed: 1.03 sec
EasyTL Acc: 78.2 %
kitchen - books
Time Elapsed: 1.29 sec
EasyTL Acc: 79.4 %
kitchen - dvd
Time Elapsed: 1.26 sec
EasyTL Acc: 79.4 %
kitchen - elec
Time Elapsed: 1.09 sec
EasyTL Acc: 79.4 %


### Office-Caltech dataset

In [ ]:
# Not implemented yet

### Image-CLEF DA and Office-Home datasets

In [96]:
# Not implemented yet